In [5]:
### LLM(Large language model) 호출
from langchain.llms import OpenAI # text-davinci-003
from langchain.chat_models import ChatOpenAI # gpt-3.5-turbo

# llm = OpenAI()
# result = llm.predict("태양계 행성은 총 몇개가 있어?")

# temperature=> 얼마나 창의적 + 무작위적 으로 하는가에 대한 수치 높을 수록 더 창의적 이고 무작위 적으로 생성됨.
chat = ChatOpenAI(temperature=0.1)
result = chat.predict("태양계 행성은 총 몇개가 있어?")

result

'태양계 행성은 총 8개가 있습니다. 이는 수성, 금성, 지구, 화성, 목성, 토성, 천왕성, 해왕성입니다.하지만 명왕성은 2006년에 왜곡체로 지정되어 행성에서 제외되었습니다.'

In [8]:
# Predict Messages
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="당신은 지리 전문가입니다. 그리고 당신은 한글으로만 답변합니다."),
    AIMessage(content="안녕하세요, 제 이름은 파올로입니다!"),
    HumanMessage(content="멕시코와 태국 사이의 거리는 어느정도 되나요? 그리고 당신의 이름은 무엇인가요?")
]
chat.predict_messages(messages)


AIMessage(content='멕시코와 태국 사이의 직선 거리는 약 15,000km 정도입니다. 제 이름은 파올로입니다.')

In [10]:
# Prompt Templates
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}")

prompt = template.format(country_a="Korea", country_b="Japan")
# chat.predict(prompt)

template = ChatPromptTemplate.from_messages([
    ("system",
     "You are a geography expert. And you only reply in {language}."),
    ("ai", "Hello, my name is {name}"),
    ("human",
     "What is the distamce between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Korea",
    name="mumusic",
    country_a="Japan",
    country_b="Korea",
)
chat.predict_messages(prompt)

AIMessage(content='일본과 한국 사이의 거리는 대략 900km입니다. 제 이름은 무뮤식입니다.')

# OutputParser and LCEL
### 사용하는 이유
- LLM의 응답을 변형해야 할 떄가 있기 떄문임.
- LLM은 항상 텍스트로만 응답하기에 응답을 변환 시켜주는 로직이 필요함.

In [30]:
# OutputParser and LCEL
from langchain.chat_models import ChatOpenAI
from langchain.schema import BaseOutputParser  # base class 같은거임. 변형해서 사용 가능함.


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        # strip => 공백 제거
        items = text.split(",")
        return list(map(str.strip, items))


parser = CommaOutputParser()
parser.parse("a, b, c ")

template = ChatPromptTemplate.from_messages([
    ("system",
     "당신은 list 생성 기계 입니다. 입력받은 질문들은 모두 comma seperated list로 답해주세요. 최대{max_items}만큼요. 전부다 소문자로 반환해주세요. list가 아닌 것으로는 답을 하지 마세요."),
    ("human", "{question}")
])

prompt = template.format_messages(
    max_items=3, question="디자인에 사용할만한 파스텔 색상을 알려주세요!")
# chat = ChatOpenAI(model="gpt-4" , temperature=0.1)
chat.predict_messages(prompt)

# -> 방향으로 return 값이 가서 마지막 값이 반환됨! chain!!
chain = template | chat | CommaOutputParser()
chain.invoke({
    "max_items":  5,
    "question": "디자인에 사용할만한 파스텔 색상을 알려주세요!"
})

['"baby pink"', '"mint green"', '"lavender"', '"peach"', '"sky blue"']